In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
pd.options.mode.chained_assignment = None
import datetime
from plotly.subplots import make_subplots
from statsmodels.tsa.tsatools import lagmat

In [12]:
df = pd.read_excel('df.xlsx')
#making the df's for PLZ sets - naming them after the lands
#dropping redundant columns at the same time
needed_columns = ['week', 'wPreis', 'full']
Sch_Hol = df[df['Plz']=="['25', '24']"][needed_columns]
NS_HB = df[df['Plz']=="['26', '27', '28']"][needed_columns]
Sch_Ham = df[df['Plz']=="['20', '21', '22', '23']"][needed_columns]
NS = df[df['Plz']=="['29', '30', '31', '38', '39']"][needed_columns]
Ber_Bra = df[df['Plz']=="['10', '11', '12', '13', '14', '15', '16']"][needed_columns]
Sach = df[df['Plz']=="['1', '4', '6', '7', '8', '9']"][needed_columns]
NR_W = df[df['Plz']=="['40', '41', '42', '44', '45', '46', '47']"][needed_columns]

#adding a datetime column and dropping the week column
lander_dfs = [Sch_Hol, NS_HB, Sch_Ham, NS, Ber_Bra, Sach, NR_W]
for df in lander_dfs:
    df.loc[:,'Datetime'] = df.week.apply(lambda x: datetime.datetime.strptime(x+'-3', "%Y-%W-%w"))
    df.drop('week', inplace = True, axis=1)

The next cell is about importing the data on construction permits number (per land per month). It is taken from Statistik der Baugenehmigungen (code 31111) placed on https://www-genesis.destatis.de/. This variable was taken because it was the only one available with monthly frequency and per land.
The Construction.xlsx file has been created MANUALLY using the table formed by the website.

In [15]:
#reading the df
baudf = pd.read_excel('Construction.xlsx').reset_index(drop=True)

#creating a months column
months = {
    'Januar': 1,
    'Februar': 2,
    'März': 3,
    'April': 4,
    'Mai': 5,
    'Juni': 6,
    'Juli': 7,
    'August': 8,
    'September': 9,
    'Oktober': 10,
    'November': 11,
    'Dezember': 12  
}
baudf['Month_number']=baudf['Month'].apply(lambda x: months[x])

#creating a week (YYYY-WW) column
baudf['Datetime']=(baudf['Year'].apply(str)+'-'+baudf['Month_number'].apply(str)+'-'+'28').apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))#.dt.isocalendar().week

#dropping redundant columns
baudf = baudf[['Land', 'Datetime', 'Value']]

In [16]:
#introduction of the functions to be used

def cutting_dfs(waste_price_df, construction_df):
    """Cuts the waste_price_df and construction_df so that their time series overlap and have the same length"""
    
    #reseting the indexes
    waste_price_df.reset_index(inplace=True, drop=True)
    construction_df.reset_index(inplace=True, drop=True)
   
    #getting and applying the window where the timeseries overlap
    earliest_date = waste_price_df.Datetime[0]
    latest_date = construction_df.Datetime.iloc[-6]
    waste_price_df = waste_price_df[(waste_price_df['Datetime']>=earliest_date)&(waste_price_df['Datetime']<=latest_date)]
    construction_df = construction_df[(construction_df['Datetime']>=earliest_date)&(construction_df['Datetime']<=latest_date)]
    construction_df.loc[:, 'Value'] = construction_df['Value'].astype('float32')

    return [waste_price_df, construction_df]

def same_length(waste_price_df, construction_df):
    """Forces the relevant timeseries to be of the same length"""

    #getting the month and year colums for waste_price_df
    waste_price_df.loc[:, 'Month'] = waste_price_df['Datetime'].apply(lambda x: x.month)
    waste_price_df.loc[:, 'Year'] = waste_price_df['Datetime'].apply(lambda x: x.year)
    #averaging the waste_price_df values by months and years
    waste_price_df = waste_price_df.groupby(['Month', 'Year', 'full']).mean().reset_index().sort_values(['full', 'Year', 'Month']).reset_index(drop=True)
    
    #sorting the average values by the waste type and creating a list of 
    if waste_price_df['full'].nunique()>1:
        waste_type_dfs = [waste_price_df[waste_price_df['full']==unique] for unique in waste_price_df['full'].unique()]
    else:
        waste_type_dfs = [waste_price_df]
       
    #considering a case when the waste_price_df has a row missing (Ber_Bra misses December'22)
    if waste_type_dfs[0].shape[0] != construction_df.shape[0]:
        #print(construction_df)
        construction_df.drop(index = 35, inplace=True)
      
    return [waste_type_dfs, construction_df]

def difference(waste_type_dfs, construction_df):
    """Helps to deal with differenced values"""
    waste_price_diff_dfs = []
    for waste_type_df in waste_type_dfs:
        waste_price_diff = np.diff(waste_type_df['wPreis'])
        waste_type_df = waste_type_df.iloc[1:, :]
        waste_type_df.loc[:, 'wPreis'] = waste_price_diff
        waste_price_diff_dfs.append(waste_type_df)

    construction_diff = np.diff(construction_df['Value'])
    construction_df = construction_df.iloc[1:, :]
    construction_df.loc[:, 'Value'] = construction_diff

    return [waste_price_diff_dfs, construction_df]

def lag(construction_df, lags=1):
    """Helps to deal with lagged values"""
    construction_lagged = lagmat(construction_df['Value'].iloc[:-5], maxlag=lags, trim="forward", original='in')
    construction_df['Value'].iloc[:-5] = construction_lagged[:, lags]
    return construction_df

def correlation(waste_type_dfs, construction_df, diff = False, lags = 0):
    """Computes the correlation coefficients and prints them nicely"""
    for waste_type_df in waste_type_dfs:
        cc = waste_type_df['wPreis'].reset_index(drop=True).corr(construction_df['Value'].reset_index(drop=True), method='spearman')
        if diff:
            print(f'Corr. coeff. between the {waste_type_df.full.unique()[0]} differences and the differences of numbers of construction permits issues is', round(cc, 2))
        elif lags > 0:
            print(f'Corr. coeff. between the {waste_type_df.full.unique()[0]} prices and the lag-{lags} numbers of construction permits issues is', round(cc, 2))
        else:
            print(f'Corr. coeff. between the {waste_type_df.full.unique()[0]} prices and the number of construction permits issues is', round(cc, 2))

def compute_correlation(waste_price_df, construction_df, diff = False, lags = 0):
    """Combines the above functions"""
    cut_dfs =  cutting_dfs(waste_price_df, construction_df)
    same_length_dfs = same_length(cut_dfs[0], cut_dfs[1])

    if diff:
       same_length_dfs = difference(same_length_dfs[0], same_length_dfs[1])
    if lags > 0:
        same_length_dfs = [same_length_dfs[0], lag(same_length_dfs[1], lags = lags)]
      
    waste_type_dfs, construction_df = same_length_dfs[0], same_length_dfs[1]
    correlation(waste_type_dfs, construction_df, diff=diff, lags=lags)

The following cell is huge, so here's a brief description of what will be going on there.
So far we've got the dataframes corresponding to each PLZ collection. For each PLZ collection we have a number of waste types (from 1 to 3).

So for each PLZ collection we:

Plot 1:
1. Plot prices of each waste type;
2. Plot numbers of construction permits issued;
3. Compute correlations of the plotted time series.

Plot 2:

4. Plot DIFFERENCES of prices of each waste type;
5. Plot DIFFERENCES of numbers of construction permits issued;
6. Compute correlations of the plotted time series.

Plot 3:

7. Plot prices of each waste type;
8. Plot LAGGED VERSIONS of numbers of construction permits issued;
9. Compute correlations of the plotted time series.

In [17]:
#massive plotting here
#first we define the lands corresponding to each of the PLZ sets
#the order is just as that of lander_df
land_names = ['Schleswig-Holstein', ['Niedersachsen', 'Bremen'], ['Schleswig-Holstein','Hamburg' ],'Niedersachsen',['Berlin', 'Brandenburg'],'Sachsen', 'Nordrhein-Westfalen']

#we will plot on by-land basis
for land_df, land_name in zip(lander_dfs, land_names):
    #merging the construction permits numbers if more than one land involved and filtering the data
    if len(land_name)==2:
        construction_df = baudf[baudf['Land'] ==land_name[0]].merge(baudf[baudf['Land'] ==land_name[1]], on='Datetime', how='outer') 
        construction_df['Value'] = construction_df['Value_x'] + construction_df['Value_y']
        construction_df = construction_df[['Datetime','Value']]
    #or just filtering the data if there is the only land involved
    else: 
        construction_df = baudf[baudf['Land'] == land_name]
        construction_df = construction_df[['Datetime','Value']]
    
    #here come plots of actual prices vs number of permits
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    #plot all types of waste for the given land
    for waste_type in land_df.full.unique():
        fig.append_trace(
            go.Scatter(x=land_df[land_df['full']==waste_type]['Datetime'], y=land_df[land_df['full']==waste_type]['wPreis'], 
                       name=f"{waste_type}"),
            row=1, col=1
        )
    #adding the plot for the construction permits in the land(s)
    fig.append_trace(
            go.Scatter(x=construction_df['Datetime'], y=construction_df['Value'], name="Construction Permits Issued"),
            row=2, col=1
        )
    #wrapping up
    fig.update_layout(title_text=f"Waste Prices in {land_name}")
    fig.show()

    #data on correlation of the above lines
    compute_correlation(land_df, construction_df)

    #here come  plots of differences in actual prices vs differences in number of permits
    #the logic is the same as with the prices themselves
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    for waste_type in land_df.full.unique():
        fig.append_trace(
            go.Scatter(x=land_df[land_df['full']==waste_type]['Datetime'], y=np.diff(land_df[land_df['full']==waste_type]['wPreis']), 
                       name=f"Difference in {waste_type} price"),
            row=1, col=1
        )
    fig.append_trace(
            go.Scatter(x=construction_df['Datetime'], y=np.diff(construction_df['Value'].iloc[:-5]), name="Difference in Construction Permits Issued"),
            row=2, col=1
        )
    fig.update_layout(title_text=f"Differences in Waste Prices in {land_name}")
    fig.show()

    #data on correlation of the above lines
    compute_correlation(land_df, construction_df, diff=True)

    #here come plots of waste prices vs LAGGED number of permits
    #we are using the lags of permits and of actual prices (not of the differences of those) since the corr. coef. look more promising
    #the logic is the same as with the prices themselves
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
    for waste_type in land_df.full.unique():
        fig.append_trace(
            go.Scatter(x=land_df[land_df['full']==waste_type]['Datetime'], y=land_df[land_df['full']==waste_type]['wPreis'], 
                       name=f"{waste_type}"),
            row=1, col=1
        )
    #adding the plot for the lagged construction permits in the land(s)
    #change the line below to change the number of lags
    #you can also make the plots lojk nicer by using trim='both'
    #but keep in mind the correlation is computed using the time series obtained by trim='forward'
    lagged = lagmat(construction_df['Value'].iloc[:-5], maxlag=10, trim="forward", original='in')
    for i in range(1, len(lagged[0])):
        fig.append_trace(
            go.Scatter(x=construction_df['Datetime'], y=lagged[:, i], name=f"Construction Permits Issued - Lag {i}"),
            row=2, col=1
        )

    #wrapping up
    fig.update_layout(title_text=f"Waste Prices in {land_name} - Compared to Lagged Construction Permits Number")
    fig.show()
    
    #data on correlation of the above lines
    for i in range(1, len(lagged[0])):
       compute_correlation(land_df, construction_df, lags=i)

Corr. coeff. between the A1 & A2 -  geschreddert prices and the number of construction permits issues is -0.68
Corr. coeff. between the A2 & A3 -  geschreddert prices and the number of construction permits issues is -0.7
Corr. coeff. between the A2 -  geschreddert prices and the number of construction permits issues is -0.68


Corr. coeff. between the A1 & A2 -  geschreddert differences and the differences of numbers of construction permits issues is 0.02
Corr. coeff. between the A2 & A3 -  geschreddert differences and the differences of numbers of construction permits issues is -0.2
Corr. coeff. between the A2 -  geschreddert differences and the differences of numbers of construction permits issues is -0.2


Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.51
Corr. coeff. between the A2 & A3 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.51
Corr. coeff. between the A2 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.48
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.35
Corr. coeff. between the A2 & A3 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.38
Corr. coeff. between the A2 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.36
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-3 numbers of construction permits issues is -0.18
Corr. coeff. between the A2 & A3 -  geschreddert prices and the lag-3 numbers of construction permits issues is -0.2
Corr. coeff. between the A2 -  geschreddert prices and the lag-3 nu

Corr. coeff. between the A2 & A3 -  geschreddert prices and the number of construction permits issues is -0.75
Corr. coeff. between the A3 -  geschreddert prices and the number of construction permits issues is -0.73


Corr. coeff. between the A2 & A3 -  geschreddert differences and the differences of numbers of construction permits issues is -0.08
Corr. coeff. between the A3 -  geschreddert differences and the differences of numbers of construction permits issues is -0.09


...

Corr. coeff. between the A1 & A2 -  geschreddert differences and the differences of numbers of construction permits issues is -0.38


Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.64
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.53
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-3 numbers of construction permits issues is -0.41
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-4 numbers of construction permits issues is -0.19
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-5 numbers of construction permits issues is -0.09
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-6 numbers of construction permits issues is -0.07
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-7 numbers of construction permits issues is 0.05
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-8 numbers of construction permits issues is 0.11
Corr. coeff. between the A1 & A2 -  geschreddert prices an

Corr. coeff. between the A1 & A2 -  geschreddert prices and the number of construction permits issues is -0.53
Corr. coeff. between the A2 & A3 -  geschreddert prices and the number of construction permits issues is -0.44
Corr. coeff. between the A3 -  geschreddert prices and the number of construction permits issues is -0.36


Corr. coeff. between the A1 & A2 -  geschreddert differences and the differences of numbers of construction permits issues is -0.31
Corr. coeff. between the A2 & A3 -  geschreddert differences and the differences of numbers of construction permits issues is -0.07
Corr. coeff. between the A3 -  geschreddert differences and the differences of numbers of construction permits issues is 0.07


Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.36
Corr. coeff. between the A2 & A3 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.27
Corr. coeff. between the A3 -  geschreddert prices and the lag-1 numbers of construction permits issues is -0.2
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.17
Corr. coeff. between the A2 & A3 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.1
Corr. coeff. between the A3 -  geschreddert prices and the lag-2 numbers of construction permits issues is -0.12
Corr. coeff. between the A1 & A2 -  geschreddert prices and the lag-3 numbers of construction permits issues is -0.03
Corr. coeff. between the A2 & A3 -  geschreddert prices and the lag-3 numbers of construction permits issues is 0.04
Corr. coeff. between the A3 -  geschreddert prices and the lag-3 numb

Overall insight is that the pattern of correlations is the same:
1. The correlation between actual waste prices and number of permits is large and negative. We assume it has something to do with the waste offer: usually the timespan between getting a construction permit and starting the construction itself is forced to be as short as possible => construction works start as soon as the permit is obtained => the waste is produced immediately => there is more waste offered on the market => the price decreases.
2. The correalion between differences of those is mainly tiny. 
3. The correlation betwee actual waste prices and lagged number of permits decreases as the number of lags increases. This happens till lag-6 or lag-7, and then the correlation increases again reaching roughly .5 at lag 10. We still do not think that this correlation is meaningful as it appears when the 0's are included into the timeseries (that is the way lagmat(...trim = 'forward') works).